In [77]:
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.indexes import VectorstoreIndexCreator

In [78]:
file = 'data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
embedding = OllamaEmbeddings(model="mxbai-embed-large:latest")

In [80]:
index = VectorstoreIndexCreator(
    embedding=embedding,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [44]:
llm = ChatOllama(model="llama3.1", temperature=0)

In [45]:
# Now manually create the QA chain to use Ollama
retriever = index.vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [46]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [47]:
response = qa_chain.run(query)

/tmp/ipykernel_95589/3841874701.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


In [48]:
display(Markdown(response))

Here is the list of shirts with sun protection:

| **Product ID** | **Name** | **Description** |
| --- | --- | --- |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | Lightweight, wrinkle-free shirt with UPF 50+ coverage and cape venting for ventilation. |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | Traditional fit shirt with UPF 50+ protection, front and back cape venting, and two front pockets. |
| 709 | Sunrise Tee (Women's) | Lightweight, high-performance fabric that wicks away moisture and dries quickly, with built-in sun protection. |
| 87 | Women's Tropical Plaid Shirt | Flattering fit shirt with UPF 50+ rated fabric, cape venting, and two front pockets for ventilation and storage. |

Note: All of these shirts have UPF 50+ or SPF 50+ rating, which means they block at least 98% of the sun's harmful UV rays.

In [49]:
loader = CSVLoader(file_path=file)

In [50]:
docs = loader.load()

In [51]:
docs[0]

Document(metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [54]:
from ollama_embed_utils import OllamaEmbeddingFunction
embeddings = OllamaEmbeddingFunction()

In [55]:
embed = embeddings.embed_query("Hi, my name is Ali")

In [56]:
print(len(embed))

1024


In [57]:
print(embed[:5])

[-0.2773186266422272, -0.010408230125904083, -0.1603201925754547, 0.2309749871492386, 0.12020942568778992]


In [59]:
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)

In [60]:
query = "Please suggest a shirt with sunblocking"


In [61]:
docs = db.similarity_search(query)

In [62]:
len(docs)

4

In [63]:
docs[0]

Document(metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 374}, page_content=": 374\nname: Men's Plaid Tropic Shirt, Short-Sleeve\ndescription: Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offers UPF 50+ coverage and is great for extended travel. SunSmart technology blocks 98% of the sun's harmful UV rays, while the high-performance fabric is wrinkle-free and quickly evaporates perspiration. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. Additional features include front and back cape venting, two front bellows pockets and an imported design. With UPF 50+ coverage, you can limit sun exposure and feel secure with the highest rated sun protection available.")

In [64]:
retriever = db.as_retriever()

In [65]:
llm = ChatOllama(temperature=0.0, model='llama3.1')

In [66]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [67]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 

/tmp/ipykernel_95589/720143539.py:1: LangChainDeprecationWarning: The method `BaseChatModel.call_as_llm` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.call_as_llm(f"{qdocs} Question: Please list all your \


In [68]:
display(Markdown(response))

Here are the shirts with sun protection listed in a table in markdown:

| **Shirt Name** | **Description** | **Fabric** | **Care Instructions** |
| --- | --- | --- | --- |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ coverage for extended travel and fishing | 52% polyester, 48% nylon | Machine washable and dryable |
| Tropical Breeze Shirt | Lightweight, breathable long-sleeve shirt with SunSmart protection | 71% nylon, 29% polyester | Machine wash and dry |
| Sunrise Tee | Women's UV-protective button down shirt for hot weather | 71% nylon, 29% polyester | Machine wash and dry |
| Sun Shield Shirt by [Brand] | High-performance sun shirt with SPF 50+ protection | 78% nylon, 22% Lycra Xtra Life fiber | Handwash, line dry |

Here's a summary of each shirt:

* **Men's Plaid Tropic Shirt, Short-Sleeve**: A lightweight, breathable short-sleeve shirt designed for fishing and travel. Offers UPF 50+ coverage and features SunSmart technology.
* **Tropical Breeze Shirt**: A long-sleeve shirt with superior SunSmart protection from the sun's harmful rays. Features wrinkle-resistant and moisture-wicking fabric.
* **Sunrise Tee**: A women's button down shirt designed for hot weather. Offers UV-protective UPF 50+ coverage and features a lightweight, high-performance fabric.
* **Sun Shield Shirt by [Brand]**: A high-performance sun shirt with SPF 50+ protection that blocks 98% of the sun's harmful rays. Features abrasion-resistant fabric and is recommended by The Skin Cancer Foundation.

All four shirts offer UPF 50+ or SPF 50+ protection, making them suitable for outdoor activities in sunny conditions.

In [69]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [70]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [71]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [73]:
display(Markdown(response))

Here is the list of shirts with sun protection:

| **Product ID** | **Name** | **Description** |
| --- | --- | --- |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | Lightweight, wrinkle-free shirt with UPF 50+ coverage and high-performance fabric that wicks away perspiration. |
| 709 | Sunrise Tee | Women's UV-protective button down shirt with lightweight, high-performance fabric that wicks away moisture and dries quickly. |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with traditional fit, wrinkle-resistant fabric, and UPF 50+ sun protection. |
| 87 | Women's Tropical Plaid Shirt | Flattering fit, lightweight shirt with UPF 50+ rated fabric that wicks away perspiration and dries quickly. |

Note: All of these shirts have UPF 50+ or SPF 50+ rating, which means they block at least 98% of the sun's harmful UV rays.